*****Installations and imports*****


In [2]:
!pip install torch torchvision
!pip install transformers==2.5.1
!pip install pandas
!pip install numpy
!pip install spacy-entity-linker
!python -m spacyEntityLinker "download_knowledge_base"
!pip install torch


     |████████████████████████████████| 501kB 5.6MB/s 
     |████████████████████████████████| 133kB 16.8MB/s 
     |████████████████████████████████| 870kB 16.0MB/s 
     |████████████████████████████████| 5.6MB 10.6MB/s 
     |████████████████████████████████| 1.2MB 43.8MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 7.4MB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=cc1d706f48e62d58b6e96f13b380fcb03ca517291efdd59882ebb8bf532a9f58
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
ERROR: botocore 1.20.45 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
!gsutil cp gs://boolq/train.jsonl .
!gsutil cp gs://boolq/dev.jsonl .

Copying gs://boolq/train.jsonl...
/ [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      
Copying gs://boolq/dev.jsonl...
/ [1 files][  2.1 MiB/  2.1 MiB]                                                
Operation completed over 1 objects/2.1 MiB.                                      


In [178]:
import random
import torch
import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
import torch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from spacyEntityLinker import EntityLinker
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score



In [5]:
# Loading data
train_data_df = pd.read_json("/content/train.jsonl", lines=True, orient='records')
dev_data_df = pd.read_json("/content/dev.jsonl", lines=True, orient="records")

In [6]:
dev_data_df

,question,title,answer,passage
0,does ethanol take more energy make that produces,Ethanol fuel,False,All biomass goes through at least some of thes...
1,is house tax and property tax are same,Property tax,True,Property tax or 'house tax' is a local tax on ...
2,is pain experienced in a missing body part or ...,Phantom pain,True,Phantom pain sensations are described as perce...
3,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts,True,Harry Potter and the Escape from Gringotts is ...
4,is there a difference between hydroxyzine hcl ...,Hydroxyzine,True,Hydroxyzine preparations require a doctor's pr...
...,...,...,...,...
3265,is manic depression the same as bi polar,Bipolar disorder,True,"Bipolar disorder, previously known as manic de..."
3266,was whiskey galore based on a true story,SS Politician,True,SS Politician was an 8000-ton cargo ship owned...
3267,are there plants on the international space st...,Plants in space,True,Plant research continued on the International ...
3268,does the hockey puck have to cross the line to...,Goal (ice hockey),True,"In ice hockey, a goal is scored when the puck ..."


In [7]:
passages_train = train_data_df.passage.values
questions_train = train_data_df.question.values
answers_train = train_data_df.answer.values.astype(int)
passages_dev = dev_data_df.passage.values
questions_dev = dev_data_df.question.values
answers_dev = dev_data_df.answer.values.astype(int)

In [8]:
print(len(questions_train))

9427


In [86]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


***Loading of train data with entailment***

In [87]:
#selecting the important question words that is verb or noun or the starting question word
#This is only for selecting the sentences from the passages.
array = []
for sent in questions_train[0:9427]:
    rows = []
    question_word =""
    question_word = sent.split()[0]
    tokenize_word = word_tokenize(sent)
    tagged = nltk.pos_tag(tokenize_word)
    for word, tag in tagged:
      if tag in ('NN', 'VB', 'VBP', 'VBZ'):
        rows.append(word)
        rows.append(word.capitalize())
      
    array.append(rows)
print(array[3])

#This will be used for making features from the questions and passages.
array_ques = []
for sent in questions_train[0:9427]:
    rows = []
    question_word =""
    question_word = sent.split()[0]
    tokenize_word = word_tokenize(sent)
    tagged = nltk.pos_tag(tokenize_word)
    for word, tag in tagged:
      if tag in ('NN', 'VB') or (word == sent.split()[0]):
        rows.append(word)
        rows.append(word.capitalize())
    
    array_ques.append(rows)
print(array_ques[3])


double_passage_line = []
for i in range(0,9427):
  passage_line = ""
  sentences = sent_tokenize(passages_train[i])
  for sent in sentences:
    if any(word in sent for word in array[i]):
      passage_line += sent
  #print(passage_line)
  double_passage_line.append(passage_line)

print(len(double_passage_line))

rows, cols = (9427, 2) 

arr=[]
for i in range(rows):
  row = []
  for j in range(cols):
    if j==0:
      tokenize_word = word_tokenize(questions_train[i])
      tagged = nltk.pos_tag(tokenize_word)
      row.append(tagged)
    else:
      #tokenize_word = double_passage_line[i]
      tokenize_word = word_tokenize(double_passage_line[i])
      #tagged = nltk.pos_tag(tokenize_word)
      row.append(tokenize_word)
  arr.append(row)

['is', 'Is', 'sugar', 'Sugar', 'sugar', 'Sugar']
['is', 'Is', 'sugar', 'Sugar', 'sugar', 'Sugar']
9427


In [66]:

#$#
#selection of passage lines with the given shortlisted words from question
double_passage_line = []
for i in range(0,9427):
  passage_line = ""
  sentences = sent_tokenize(passages_train[i])
  for sent in sentences:
    if any(word in sent for word in array[i]):
      passage_line += sent
  #print(passage_line)
  double_passage_line.append(passage_line)

print(len(double_passage_line))
  

9427


In [67]:

#$#
rows, cols = (9427, 2) 

arr=[]
for i in range(rows):
  row = []
  for j in range(cols):
    if j==0:
      tokenize_word = word_tokenize(questions_train[i])
      tagged = nltk.pos_tag(tokenize_word)
      row.append(tagged)
    else:
      #tokenize_word = double_passage_line[i]
      tokenize_word = word_tokenize(double_passage_line[i])
      #tagged = nltk.pos_tag(tokenize_word)
      row.append(tokenize_word)
  arr.append(row)

  #print(arr[0:5])

***Features***

In [88]:
#function for checking the negative question verb 
def negation_check(first_word, sentence):
  #print(first_word)
  #print(sentence)
  val = 0
  
  for word in sentence:
    #print(word)
    if first_word == " will " and (word == "won't" or word == "Won't") :
      #print("1","present")
      val = 1
      break
    elif first_word == " can " and (word == "can't" or word == "Can't") :
      #print("1","present")
      val = 1
      break
    
    elif first_word == " do " and (word == "don't" or word == "Don't") :
      #print("1","present")
      val = 1
      break
  
    elif first_word == " are " and (word == "aren't" or word == "Aren't") :
      #print("1","present")
      val = 1
      break

    elif first_word == " does " and (word == "doesn't" or word == "Doesn't"):
      #print("2","present")
      val = 1
      break

    elif first_word == " is " and (word == "isn't" or word == "Isn't"):
      #print("3","present")
      val = 1
      break

    elif first_word == " was " and (word == "wasn't" or word == "Wasn't"):
      #print("5","present")
      val = 1
      break

    elif first_word == " were " and (word == "weren't" or word == "Weren't"):
      #print("6","present")
      val = 1
      break

    elif word == sentence[len(sentence)-1]:
      #print("not present")
      val = 0
      
    else:
      continue
  return val

In [89]:
#function checking for not in the passage
def not_check(sentence):
  val = 0
  for word in sentence:
    
    if word == "not":
      val = 1
      break
  return val


In [90]:
#function checking for verbs that are present in question are there in answer or not
def check_for_verb(verb, sentence):
  verb_present = 0
  for word, tag in verb[1:]:
    #print(word, tag)
    if tag == 'VBZ' or tag == 'VB' or tag == 'VBP':
      if word in sentence:
        verb_present = 1
        #print("verb is there")
        break
  return verb_present
  #print(sentence)

In [91]:
#checking whether last noun in question is present in the passage or not?
def check_for_last_noun(last_tuple, sentence):
  #print(last_tuple[1])
  #print(sentence)
  last_noun_present = 0
  if last_tuple[1] == 'NN' or last_tuple[1] == 'NNS' or last_tuple[1] == 'NNP':
    if last_tuple[0] in sentence:
      last_noun_present = 1
  return last_noun_present
  #print(sentence)

In [92]:
#function checking for similar verbs
def check_similar_verb(verb_sent, sentence):
  #print("verb_sent", verb_sent)
  #print(sentence)
  val = 0
  for word, tag in verb_sent:
    if tag == 'VBP':
      synonyms = [] 
      for syn_set in wordnet.synsets(word): 
        for l in syn_set.lemmas(): 
          synonyms.append(l.name()) 
      if any(word in sentence for word in synonyms):
        #print("present")
        val = 1
  return val
        


In [93]:
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [94]:
print(questions_train[1])
print(passages_train[1])
print(answers_train[1])

do good samaritan laws protect those who help at an accident
Good Samaritan laws offer legal protection to people who give reasonable assistance to those who are, or who they believe to be, injured, ill, in peril, or otherwise incapacitated. The protection is intended to reduce bystanders' hesitation to assist, for fear of being sued or prosecuted for unintentional injury or wrongful death. An example of such a law in common-law areas of Canada: a good Samaritan doctrine is a legal principle that prevents a rescuer who has voluntarily helped a victim in distress from being successfully sued for wrongdoing. Its purpose is to keep people from being reluctant to help a stranger in need for fear of legal repercussions should they make some mistake in treatment. By contrast, a duty to rescue law requires people to offer assistance and holds those who fail to do so liable.
1


In [96]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    text = str(text)
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    #print("tokenized text", tokenized_text)
    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
    #print('list tokens', list_token_embeddings)
    return list_token_embeddings

In [97]:
def target_embedding(texts):
  target_word_embeddings = []
  #for text in texts:
    #print(text)
  for text in texts:
    
      tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    
    
      list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
      tokenized_texter = []

      # Find the position 'bank' in list of tokens
      for element in tokenized_text:
      
        tokenized_texter.append(element.capitalize())

      #print("text", text)
      if text == None:
        continue
      else:
        text_split = text.split()
        f = 0
        for i in text_split:
          if i in tokenized_text :
            f=0
            word_index = tokenized_text.index(i)
          elif i in tokenized_texter :
            f=0
            word_index = tokenized_texter.index(i)
          else:
            f=1
            continue
        if f==1:
          continue
    
      #print("text", text, "tokenized_text", tokenized_text, "tokens_tensor", tokens_tensor, "segments_tensors", segments_tensors,"word index", word_index)
      word_embedding = list_token_embeddings[word_index]
      target_word_embeddings.append(word_embedding)
  return target_word_embeddings


In [98]:
entityLinker = EntityLinker()

#initialize language model
nlp = spacy.load("en_core_web_sm")

#add pipeline
nlp.add_pipe(entityLinker, last=True, name="entityLinker")


In [99]:
def entity_linked(train_question, train_passage):
  
  #linking the entity with spacy and returning the percentage of matched ids.
  question_entity = []
  answer_entity = []
  question_entity_names = []
  passage_entity_names = []
  question = nlp(train_question)#questions_train[1]
  passage = nlp(train_passage)
  all_linked_entities_question=question._.linkedEntities
  all_linked_entities_passage=passage._.linkedEntities
  i=0

  for sent in question.sents:
    j=0
    for j in sent._.linkedEntities:
      question_entity.append(j.get_id())
      #print("names",j.get_label())
      question_entity_names.append(j.get_label())

  for sent in passage.sents:
    j=0
    for j in sent._.linkedEntities:
      answer_entity.append(j.get_id())
      #print("names in passages",j.get_label())
      passage_entity_names.append(j.get_label())
      passage_entity_names = list(set(passage_entity_names))
    #sent._.linkedEntities.pretty_print()

  count=0
  for i in question_entity:
    if i in question_entity and i in answer_entity:
      count+=1
  if len(question_entity)==0:
    percent = 0
  else:
    percent = count/len(question_entity)
  

  #using the linked entities and applying bert embeddings on that
  texts = passage_entity_names
  texting = question_entity_names
  question_embedding = target_embedding(texting)
  passage_embedding = target_embedding(texts)

  # Calculating the distance between the
  # embeddings of 'bank' in all the
  # given contexts of the word

  list_of_distances = []
  for text1, embed1 in zip(texting, question_embedding):
      #print("first loop",text1,embed1)
      for text2, embed2 in zip(texts, passage_embedding):
          #print("second loop",text1,embed1)
          cos_dist = 1 - cosine(embed1, embed2)
          list_of_distances.append([text1,embed1,text2, embed1, cos_dist])

  distances_df = pd.DataFrame(list_of_distances, columns=['text1','embedding1','text2','embedding2','similarity'])

  similar_list = []
  cosine_list = []
  similarity_df = distances_df[distances_df.similarity > 0.65]
  cosine_list = list(set(similarity_df['text1']))

  if len(question_entity_names)==0:
    feature_result=0
  else:
    feature_result = len(cosine_list)/len(question_entity_names) 
  #feature_result
  #print("linked",round(percent,2), "linked with bert", round(feature_result,2))
  return round(percent,2), round(feature_result,2)

In [101]:
#function for making the data using the given set of features
def features(value, index, train_question, train_passage):
    """Set of four features from lecture"""
    
    #print("0th value",value[1])
    negation = 0
    negation = negation_check(value[0][0][0], value[1])
    #feature2 checking for not
    not_word = 0
    not_word = not_check(value[1])
    #checking for VB present in ans or not.
    verb_check = 0
    verb_check = check_for_verb(value[0], value[1])
    #checking for the last noun
    last_noun = 0
    ques_len = len(value[0])
    last_noun = check_for_last_noun(value[0][ques_len-1], value[1])
    
    #check for similarity between the verbs present in question and answer
    similar_verb = 0
    if (verb_check==0):
      similar_verb = check_similar_verb(value[0], value[1])
    #checking for all the nouns present in the passage or not
    all_nouns = 0
    #linked = entity id's matched percentage
    #linked_with_bert = linked entities with bert and cosine similarity percentage
    linked, linked_with_bert = entity_linked(train_question, train_passage)

    #bias
    f1 = 1 if negation is 1 else 0
    f2 = 1 if not_word is 1 else 0
    f3 = 1 if verb_check is 1 else 0
    f4 = 1 if last_noun is 1 else 0
    f5 = 1 if similar_verb is 1 else 0
    f6 = linked
    f7 = linked_with_bert
    f8 = 1
    return np.array([f1, f2, f3, f4, f5, f6, f7,f8])    


features_matrix = [[0,0,0,0,0,0,0,0]]
for i in range(5):
    a = np.array(features_matrix)
    b = np.array([features(arr[i],i, questions_train[i], passages_train[i])])
    if i%100 == 0:
      print(i)

    features_matrix = np.concatenate((a,b), axis=0)
features_matrix = features_matrix[1:]


0


***Loading the data for test set***

In [47]:
#selecting the important question words that is verb or noun or the starting question word
#This is only for selecting the sentences from the passages.
array = []
for sent in questions_dev[0:3270]:
    rows = []
    question_word =""
    question_word = sent.split()[0]
    tokenize_word = word_tokenize(sent)
    tagged = nltk.pos_tag(tokenize_word)
    for word, tag in tagged:
      if tag in ('NN', 'VB', 'VBP', 'VBZ'):
        rows.append(word)
        rows.append(word.capitalize())

    
    array.append(rows)
print(array[3])

#This will be used for making features from the questions and passages.
array_ques = []
for sent in questions_dev[0:3270]:
    rows = []
    question_word =""
    question_word = sent.split()[0]
    tokenize_word = word_tokenize(sent)
    tagged = nltk.pos_tag(tokenize_word)
    for word, tag in tagged:
      if tag in ('NN', 'VB') or (word == sent.split()[0]):
        rows.append(word)
        rows.append(word.capitalize())

    
    array_ques.append(rows)
print(array_ques[3])

#selection of passage lines with the given shortlisted words from question
double_passage_line = []
for i in range(0,3270):
  passage_line = ""
  sentences = sent_tokenize(passages_dev[i])
  for sent in sentences:
    if any(word in sent for word in array[i]):
      passage_line += sent
  #print(passage_line)
  double_passage_line.append(passage_line)

print(len(double_passage_line))

rows, cols = (3270, 2) 

arr=[]
for i in range(rows):
  row = []
  for j in range(cols):
    if j==0:
      tokenize_word = word_tokenize(questions_dev[i])
      tagged = nltk.pos_tag(tokenize_word)
      row.append(tagged)
    else:
      #tokenize_word = double_passage_line[i]
      tokenize_word = word_tokenize(double_passage_line[i])
      #tagged = nltk.pos_tag(tokenize_word)
      row.append(tokenize_word)
  arr.append(row)


['is', 'Is', 'potter', 'Potter', 'escape', 'Escape', 'gringotts', 'Gringotts', 'roller', 'Roller', 'coaster', 'Coaster', 'ride', 'Ride']
['is', 'Is', 'potter', 'Potter', 'escape', 'Escape', 'gringotts', 'Gringotts', 'roller', 'Roller', 'coaster', 'Coaster', 'ride', 'Ride']


In [48]:

#$#
#selection of passage lines with the given shortlisted words from question
double_passage_line = []
for i in range(0,3270):
  passage_line = ""
  sentences = sent_tokenize(passages_dev[i])
  for sent in sentences:
    if any(word in sent for word in array[i]):
      passage_line += sent
  #print(passage_line)
  double_passage_line.append(passage_line)

print(len(double_passage_line))

3270


In [50]:
#$#
rows, cols = (3270, 2) 

arr=[]
for i in range(rows):
  row = []
  for j in range(cols):
    if j==0:
      tokenize_word = word_tokenize(questions_dev[i])
      tagged = nltk.pos_tag(tokenize_word)
      row.append(tagged)
    else:
      #tokenize_word = double_passage_line[i]
      tokenize_word = word_tokenize(double_passage_line[i])
      #tagged = nltk.pos_tag(tokenize_word)
      row.append(tokenize_word)
  arr.append(row)

***Loading feature matrix for test set***

In [51]:
feature_matrix_test = [[0,0,0,0,0,0,0,0]]
for i in range(0,3270):
    a = np.array(feature_matrix_test)
    b = np.array([features(arr[i],i, questions_dev[i], passages_dev[i])])
    if i%100 == 0:
      print(i)
    feature_matrix_test = np.concatenate((a,b), axis=0)
feature_matrix_test = feature_matrix_test[1:]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


***Bidirectional model without bert embeddings***

In [199]:
#model5 is the Bidirectional Lstm applied with 1 embedding layer, 2 bidirectional layers and 1 output layer
inputs5 = keras.Input(shape=(8,), dtype="int32")
x5 = layers.Embedding(9427, 8)(inputs5)
x5 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x5)
x5 = layers.Bidirectional(layers.LSTM(50))(x5)
outputs5 = layers.Dense(2, activation="softmax")(x5)
model5 = keras.Model(inputs5, outputs5)
model5.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 8, 8)              75416     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 8, 100)            23600     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 159,618
Trainable params: 159,618
Non-trainable params: 0
_________________________________________________________________


In [200]:
model5.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [201]:
model5.fit(
  feature_matrix,
  to_categorical(answers_train),
  epochs=15,
  batch_size=32
)

Epoch 1/15
295/295 [==============================] - 13s 20ms/step - loss: 0.6656 - accuracy: 0.6208
Epoch 2/15
295/295 [==============================] - 6s 21ms/step - loss: 0.6642 - accuracy: 0.6184
Epoch 3/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6621 - accuracy: 0.6209
Epoch 4/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6597 - accuracy: 0.6210
Epoch 5/15
295/295 [==============================] - 6s 21ms/step - loss: 0.6579 - accuracy: 0.6250
Epoch 6/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6587 - accuracy: 0.6238
Epoch 7/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6570 - accuracy: 0.6246
Epoch 8/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6590 - accuracy: 0.6181
Epoch 9/15
295/295 [==============================] - 6s 21ms/step - loss: 0.6560 - accuracy: 0.6226
Epoch 10/15
295/295 [==============================] - 6s 20ms/step - loss: 0.6562 - accur

In [202]:
model5.evaluate(
  feature_matrix_test,
  to_categorical(answers_dev)
)

103/103 [==============================] - 2s 4ms/step - loss: 0.6566 - accuracy: 0.6297


[0.6565719842910767, 0.6296635866165161]

In [203]:
predictions5 = model5.predict(feature_matrix_test)

In [204]:
y_pred = np.argmax(predictions5, axis=1)
print(y_pred[0:20])
print(answers_dev[0:20])

[1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [205]:
print(f1_score(answers_dev, y_pred, average='weighted'))
print(accuracy_score(answers_dev, y_pred))

0.5206346053618371
0.6296636085626911


***LSTM model without Bert Embeddings***

In [192]:
#model is the model containing 1 embedding layer, 1 lstm layer, 2 dense relu layers, 1 output layer with sigmoid activation function
model = keras.Sequential()
model.add(layers.Embedding(input_dim=9427, output_dim=64))
model.add(layers.LSTM(100))
model.add(layers.Dense(20, activation="relu"))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(2, activation="sigmoid"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          603328    
_________________________________________________________________
lstm_15 (LSTM)               (None, 100)               66000     
_________________________________________________________________
dense_8 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 22        
Total params: 671,580
Trainable params: 671,580
Non-trainable params: 0
_________________________________________________________________


In [193]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [194]:
model.fit(
  feature_matrix,
  to_categorical(answers_train),
  epochs=20,
  batch_size=16
)

Epoch 1/20
590/590 [==============================] - 12s 15ms/step - loss: 0.6677 - accuracy: 0.6211
Epoch 2/20
590/590 [==============================] - 11s 19ms/step - loss: 0.6567 - accuracy: 0.6305
Epoch 3/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6568 - accuracy: 0.6289
Epoch 4/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6583 - accuracy: 0.6236
Epoch 5/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6587 - accuracy: 0.6232
Epoch 6/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6625 - accuracy: 0.6113
Epoch 7/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6611 - accuracy: 0.6142
Epoch 8/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6565 - accuracy: 0.6224
Epoch 9/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6598 - accuracy: 0.6194
Epoch 10/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6605 - accu

In [195]:
model.evaluate(
  feature_matrix_test,
  to_categorical(answers_dev)
)

103/103 [==============================] - 1s 3ms/step - loss: 0.6573 - accuracy: 0.6217


[0.65729159116745, 0.6217125654220581]

In [196]:
predictions = model.predict(feature_matrix_test)

In [197]:
y_pred = np.argmax(prediction, axis=1)
print(y_pred[0:20])
print(answers_dev[0:20])

[0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1]
[0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [198]:
print(f1_score(answers_dev, y_pred, average='weighted'))
print(accuracy_score(answers_dev, y_pred))

0.4900197078413221
0.4850152905198777


***BiLSTM with Bert Embeddings***

In [115]:
import numpy as np
from transformers import AutoTokenizer, pipeline, TFDistilBertModel
from scipy.spatial.distance import cosine


model_bert = BertModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
pipe = pipeline('feature-extraction', model=model_bert, tokenizer=tokenizer)

In [ ]:
def transformer_embedding(name,inp,model_name):
    features = pipe(inp)
    features = np.squeeze(features)
    return features
#z=['The brown fox jumped over the dog','The ship sank in the Atlantic Ocean']
x = np.zeros((9427,21,768)) # Make a 10 by 20 by 30 array
listing = str(feature_matrix[0])
embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
x[0] = embedding_features1

listing = str(feature_matrix[1])
embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
x[1] = embedding_features1


listing = str(feature_matrix[2])
embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
if embedding_features1.shape[0]>21:
    limit = embedding_features1.shape[0]
    for i in range(limit-1,20,-1):
      embedding_features1 = np.delete(embedding_features1, i, 0)
      print(embedding_features1.shape)
x[2] = embedding_features1



listing = str(feature_matrix[3])
embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
shape = np.shape(embedding_features1)
if embedding_features1.shape[0]>21:
 
    limit = embedding_features1.shape[0]
    
    for i in range(limit-1,20,-1):
      embedding_features1 = np.delete(embedding_features1, i, 0)
elif embedding_features1.shape[0]<21:
  padded_array = np.zeros((21, 768))
  padded_array[:shape[0],:shape[1]] = embedding_features1
    
x[3] = padded_array


for j in range(4,9427):
  listing = str(feature_matrix[j])
  embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
  shape = np.shape(embedding_features1)
  if j%200==0:
    print(j)
  if embedding_features1.shape[0]>21:
      limit = embedding_features1.shape[0]
      for i in range(limit-1,20,-1):
          embedding_features1 = np.delete(embedding_features1, i, 0)
          embedding = embedding_features1
  elif embedding_features1.shape[0]<21:
      padded_array = np.zeros((21, 768))
      padded_array[:shape[0],:shape[1]] = embedding_features1
      embedding = padded_array
  else:
      embedding = embedding_features1
      
  x[j] = embedding


(21, 768)
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200


In [156]:
y = np.zeros((3270,21,768))

listing = str(feature_matrix_test[0])
embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
shape = np.shape(embedding_features1)
#if j%200==0:
    #print(j)
if embedding_features1.shape[0]>21:
    limit = embedding_features1.shape[0]
    for i in range(limit-1,20,-1):
        embedding_features1 = np.delete(embedding_features1, i, 0)
        embedding = embedding_features1
elif embedding_features1.shape[0]<21:
    padded_array = np.zeros((21, 768))
    padded_array[:shape[0],:shape[1]] = embedding_features1
    embedding = padded_array
else:
    embedding = embedding_features1
      
y[0] = embedding


for j in range(1,3270):
  listing = str(feature_matrix_test[j])
  embedding_features1=transformer_embedding('bert-base-uncased',listing,BertModel)
  shape = np.shape(embedding_features1)
  if j%200==0:
    print(j)
  if embedding_features1.shape[0]>21:
      limit = embedding_features1.shape[0]
      for i in range(limit-1,20,-1):
          embedding_features1 = np.delete(embedding_features1, i, 0)
          embedding = embedding_features1
  elif embedding_features1.shape[0]<21:
      padded_array = np.zeros((21, 768))
      padded_array[:shape[0],:shape[1]] = embedding_features1
      embedding = padded_array
  else:
      embedding = embedding_features1
      
  y[j] = embedding

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200


In [185]:
model = Sequential()
model.add(Bidirectional(LSTM(12, input_shape=( 21,768), return_sequences=True)))
model.add(LSTM(12, return_sequences=True))
model.add(LSTM(6, return_sequences=False))
model.add((Dense((2))))

In [186]:
#model.compile(optimizer='sgd', loss='mse')
model.compile(optimizer='sgd', loss ='mse', metrics=['accuracy'])
# This builds the model for the first time:
model.fit(x, answers_train, batch_size=32, epochs=10)

Epoch 1/10
295/295 [==============================] - 18s 35ms/step - loss: 0.2636 - accuracy: 0.4599
Epoch 2/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2354 - accuracy: 0.5521
Epoch 3/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2334 - accuracy: 0.5481
Epoch 4/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2347 - accuracy: 0.5148
Epoch 5/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2363 - accuracy: 0.4856
Epoch 6/10
295/295 [==============================] - 10s 36ms/step - loss: 0.2338 - accuracy: 0.4928
Epoch 7/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2343 - accuracy: 0.4842
Epoch 8/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2310 - accuracy: 0.4934
Epoch 9/10
295/295 [==============================] - 10s 35ms/step - loss: 0.2338 - accuracy: 0.4805
Epoch 10/10
295/295 [==============================] - 10s 35ms/step - loss: 0.235

In [189]:
prediction = model.predict(y)

In [190]:
y_pred = np.argmax(prediction, axis=1)
print(y_pred[0:20])
print(answers_dev[0:20])

[0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1]
[0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [191]:
print(f1_score(answers_dev, y_pred, average='weighted'))
print(accuracy_score(answers_dev, y_pred))

0.4900197078413221
0.4850152905198777


***LSTM with embeddings***

In [176]:
model = Sequential()
model.add(LSTM(12, input_shape=( 21,768), return_sequences=True))
model.add(LSTM(12, return_sequences=True))
model.add(LSTM(6, return_sequences=False))
model.add((Dense((2))))

In [177]:
#model.compile(optimizer='sgd', loss='mse')
model.compile(optimizer='adam', loss ='binary_crossentropy', metrics=['accuracy'])
# This builds the model for the first time:
model.fit(x, answers_train, batch_size=16, epochs=10)

Epoch 1/10
590/590 [==============================] - 20s 25ms/step - loss: 0.8416 - accuracy: 0.5098
Epoch 2/10
590/590 [==============================] - 15s 25ms/step - loss: 0.6624 - accuracy: 0.5096
Epoch 3/10
590/590 [==============================] - 15s 25ms/step - loss: 0.6615 - accuracy: 0.5116
Epoch 4/10
590/590 [==============================] - 16s 26ms/step - loss: 0.6639 - accuracy: 0.5084
Epoch 5/10
590/590 [==============================] - 15s 26ms/step - loss: 0.6646 - accuracy: 0.5033
Epoch 6/10
590/590 [==============================] - 15s 26ms/step - loss: 0.6674 - accuracy: 0.4942
Epoch 7/10
590/590 [==============================] - 15s 26ms/step - loss: 0.6632 - accuracy: 0.4995
Epoch 8/10
590/590 [==============================] - 16s 26ms/step - loss: 0.6650 - accuracy: 0.4948
Epoch 9/10
590/590 [==============================] - 16s 26ms/step - loss: 0.6632 - accuracy: 0.4902
Epoch 10/10
590/590 [==============================] - 16s 27ms/step - loss: 0.663

In [179]:
prediction = model.predict(y)

In [184]:
print(f1_score(answers_dev, y_pred, average='weighted'))
print(accuracy_score(answers_dev, y_pred))

0.382146790008909
0.4235474006116208
